In [2]:
! pip install -r requirements.txt --quiet

# Connecting to a Remote MCP Server with Semantic Kernel

This notebook demonstrates how to connect to a remote MCP Server using **Semantic Kernel's** `MCPStreamableHttpPlugin`. The **Model Context Protocol (MCP)** enables scalable and modular tool integration across distributed systems. 

<br/>

> **Why Use Model Context Protocol (MCP)?**
>
>MCP allows agents to discover, invoke, and manage tools dynamically across remote servers.  
>It promotes modularity, scalability, and separation of concerns, making it easier to maintain and extend AI systems as they grow in complexity.

In [19]:
from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent,ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from dotenv import load_dotenv
from os import environ
import asyncio

from semantic_kernel.connectors.mcp import MCPStreamableHttpPlugin

load_dotenv(override=True)

kernel = Kernel()

kernel.add_service(AzureChatCompletion(
    service_id="chat",
    deployment_name=environ["AZURE_OPENAI_MODEL"],
    base_url=environ["AZURE_OPENAI_ENDPOINT"],
    api_key=environ["AZURE_OPENAI_API_KEY"] ))



In [22]:

messages = [
    "Which products generated the most revenue?",
    "Drill down into customer details product with the highest revenue.",
    "Which customer had the highest sales for this product?",
]

thread: ChatHistoryAgentThread | None = None

async with MCPStreamableHttpPlugin(
    name="sales",
    url=f"{environ['MCP_SERVER_URL']}",
) as sales_plugin:

    agent = ChatCompletionAgent(
        kernel=kernel, 
        name="SalesAgent", 
        plugins=[sales_plugin, ]
    )


    for msg in messages:
        print("------------------------------------------------\n")
        print(msg)
        response = await agent.get_response(messages=msg, thread=thread)
        print(response)
        print("\n------------------------------------------------\n")

------------------------------------------------

Which products generated the most revenue?
Based on product sales revenue, here is the summary:

### Products and Revenue
1. **Brake Pad**
   - Revenue: $10,796.5
   - Category: Braking

2. **Oil Filter**
   - Revenue: $1,732.8
   - Category: Engine

3. **Spark Plug**
   - Revenue: $1,062.44
   - Category: Electrical

4. **Alternator**
   - No recorded sales in the dataset.

5. **Transmission Kit**
   - No recorded sales in the dataset.

### Most Revenue
The **Brake Pad** generated the most revenue of **$10,796.5**, dominating sales performance among all listed products.

------------------------------------------------

------------------------------------------------

Drill down into customer details product with the highest revenue.
The product with the highest revenue is **Spark Plug**, belonging to the "Electrical" category. Its unit price is $8.0, and its unit cost is $2.0.

The customer involved is **NAPA**, which operates in the